In [1]:
from pyxcel.sheet_manager import SheetManager

In [2]:
from tqdm.notebook import trange,tqdm

In [3]:
import numpy as np
import itertools
import pandas as pd
import seaborn as sns

In [4]:
workbook=SheetManager('Design_Spreadsheet.xlsx')

In [5]:
input_keys=['Aerofoil','AR','Taper Ratio','b','h']
input_locations=[('Aerodynamics','V2'),('Aerodynamics','C3'),('Aerodynamics','C7'),('Structures','C2'),('Structures','C3')]
input_cell_locations = dict(zip(input_keys,input_locations))
input_cell_locations

{'Aerofoil': ('Aerodynamics', 'V2'),
 'AR': ('Aerodynamics', 'C3'),
 'Taper Ratio': ('Aerodynamics', 'C7'),
 'b': ('Structures', 'C2'),
 'h': ('Structures', 'C3')}

In [6]:
aerofoils = workbook.get_cell('Aerofoil Data','A1:A50').value
aerofoils= list(filter(None, aerofoils))

ARs = np.arange(4,12,3)
taper_ratios = np.arange(0.25,0.61,0.2)
 
widths = np.linspace(4,10,3)*1e-2
heights = np.linspace(4,10,3)*1e-2

In [7]:
output_keys=['Battery Mass','Max Stress']
output_locations=[('Mass Breakdown','C32'),('Structures','C4')]
output_cell_locations = dict(zip(output_keys,output_locations))
output_cell_locations

{'Battery Mass': ('Mass Breakdown', 'C32'), 'Max Stress': ('Structures', 'C4')}

In [8]:
combinations=list(itertools.product(aerofoils,ARs,taper_ratios,widths,heights))
df=pd.DataFrame(combinations,columns=input_keys)
for key in output_keys:
    df[key]=None
    

In [9]:
df

,Aerofoil,AR,Taper Ratio,b,h,Battery Mass,Max Stress
0,NACA 2412,4,0.25,0.04,0.04,None,None
1,NACA 2412,4,0.25,0.04,0.07,None,None
2,NACA 2412,4,0.25,0.04,0.10,None,None
3,NACA 2412,4,0.25,0.07,0.04,None,None
4,NACA 2412,4,0.25,0.07,0.07,None,None
...,...,...,...,...,...,...,...
1561,SD7037 ( E ),10,0.45,0.07,0.07,None,None
1562,SD7037 ( E ),10,0.45,0.07,0.10,None,None
1563,SD7037 ( E ),10,0.45,0.10,0.04,None,None
1564,SD7037 ( E ),10,0.45,0.10,0.07,None,None


In [10]:
for i in trange(len(df.index)):
    for key in input_keys:
        workbook.change_cell(*input_cell_locations[key],df[key][i])
    for key in output_keys:
        df[key][i]=workbook.check_cell(*output_cell_locations[key])

C:\Users\paras\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
df

,Aerofoil,AR,Taper Ratio,b,h,Battery Mass,Max Stress
0,NACA 2412,4,0.25,0.04,0.04,0.0795432,52337.4
1,NACA 2412,4,0.25,0.04,0.07,0.0795432,17089.8
2,NACA 2412,4,0.25,0.04,0.10,0.0795432,8373.99
3,NACA 2412,4,0.25,0.07,0.04,0.0795432,29907.1
4,NACA 2412,4,0.25,0.07,0.07,0.0795432,9765.58
...,...,...,...,...,...,...,...
1561,SD7037 ( E ),10,0.45,0.07,0.07,0.0566773,9765.58
1562,SD7037 ( E ),10,0.45,0.07,0.10,0.0566773,4785.14
1563,SD7037 ( E ),10,0.45,0.10,0.04,0.0566773,20935
1564,SD7037 ( E ),10,0.45,0.10,0.07,0.0566773,6835.91
